# Impervious Surfaces

A script to calculate the % coverage of impervious surface within each CSO Small Area.

## Script Setup

In [7]:
#load the libraries
library(sf)
library(rgdal)
library(rgeos)
library(raster)
library(tidyverse)

#install these packages if they are not installed
# install.packages('geojsonio')
# install.packages('exactextractr')

library(geojsonio)
library(exactextractr)

Warning message:
"package 'exactextractr' was built under R version 3.6.3"

In [8]:
#set a directory for export 
exportDirectory <- "../1_InputData/1c_ImperviousSurface"

# Data

## Small Areas

In [9]:
#get the small area spatial data -Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015
smallAreaShp <- st_read("./downloads/d9876b9b-96eb-4f04-afec-300c5d0a276c2020328-1-wq0rek.ua66c.shp")
smallAreaShp <- smallAreaShp[c('GUID', 'COUNTYNAME')]

Reading layer `d9876b9b-96eb-4f04-afec-300c5d0a276c2020328-1-wq0rek.ua66c' from data source `C:\GitHub\ISVEHI\0_Scripts\downloads\d9876b9b-96eb-4f04-afec-300c5d0a276c2020328-1-wq0rek.ua66c.shp' using driver `ESRI Shapefile'
Simple feature collection with 18641 features and 21 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -10.66297 ymin: 51.4199 xmax: -5.996278 ymax: 55.44658
Geodetic CRS:  WGS 84


## Impervious Surface

The impervious surface data can be found at https://land.copernicus.eu/pan-european/high-resolution-layers/imperviousness/status-maps. Data from 2015 hjas been used in the ISVEHI. You will need to sign up/sign in to access this data.

Unzip the files, and place all of the extracted files into a directory. These two tiles need to be joined into one larger file.

Tile 1
https://land.copernicus.eu/pan-european/high-resolution-layers/imperviousness/status-maps/2015?tab=download&selected:list=imd-2015-20m-e30n30
Tile 2
https://land.copernicus.eu/pan-european/high-resolution-layers/imperviousness/status-maps/2015?tab=download&selected:list=imd-2015-20m-e20n30

In [15]:
# the directory of the two impervious surface TIF tiles.
imperviousDir <- 'C:/Users/jfitton/Downloads/Impervious'

In [16]:
# merge the tiles into one (make take some time to complete)
files <- list.files(path = imperviousDir, pattern = ".tif$", full.names = TRUE)
raster <- lapply(files, raster)

imperviousRaster <- do.call(merge, c(raster, tolerance = 1))

# read locally
# imperviousRaster <- raster('D:/Spatial_Data/Copernicus/LMS/IMD2015/Extracted/Processed/IMD_2015_20m_eu_03035_d05_IRL_WGS84.tif')

In [17]:
#convert to a simple feature
smallAreasSF <- st_as_sf(smallAreaShp)

#calculate the area of the raster cell (m2)
cellArea <- 20*20

#calculate the number of cells within each Small Area - uses a simple feature
smallAreasSF$impCount <- exact_extract(imperviousRaster, smallAreasSF, 'count')

#calculate the area (m2) in each small area of the raster data
smallAreasSF$impArea <- smallAreasSF$impCount*400

#calculate the area of impervious coverage in each small area
smallAreasSF$impCoverage <- (exact_extract(imperviousRaster, smallAreasSF, 'sum')/100)*cellArea

#calculate the percentage of each small area that is impervious
smallAreasSF$impPercent <- (smallAreasSF$impCoverage/smallAreasSF$impArea)*100

head(smallAreasSF)

Warning message in .local(x, y, ...):
"Polygons transformed to raster CRS (EPSG:NA)"

  |======================================================================| 100%


Warning message in .local(x, y, ...):
"Polygons transformed to raster CRS (EPSG:NA)"

  |======================================================================| 100%


GUID,COUNTYNAME,geometry,impCount,impArea,impCoverage,impPercent
4c07d11d-f71d-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-9.093292 5...,60919.08,24367633,32756.00,0.13442422
4c07d11d-f71e-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.987643 5...,30383.89,12153555,42932.04,0.35324677
4c07d11d-f71f-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.930126 5...,23336.02,9334406,41844.36,0.44828093
4c07d11d-f720-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.926795 5...,28371.98,11348791,27096.68,0.23876266
4c07d11d-f564-851d-e053-ca3ca8c0ca7f,Galway County,MULTIPOLYGON (((-9.544816 5...,203424.19,81369675,43149.14,0.05302853
4c07d11d-f565-851d-e053-ca3ca8c0ca7f,Galway County,MULTIPOLYGON (((-9.73048 53...,160154.42,64061769,24731.96,0.03860642


### Z Scores

In [18]:
#scale the percentages to z scores
smallAreasSF$impervious_pct_Z <- scale(smallAreasSF$impPercent)
head(smallAreasSF)

GUID,COUNTYNAME,geometry,impCount,impArea,impCoverage,impPercent,impervious_pct_Z
4c07d11d-f71d-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-9.093292 5...,60919.08,24367633,32756.00,0.13442422,-1.0001794
4c07d11d-f71e-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.987643 5...,30383.89,12153555,42932.04,0.35324677,-0.9907726
4c07d11d-f71f-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.930126 5...,23336.02,9334406,41844.36,0.44828093,-0.9866872
4c07d11d-f720-851d-e053-ca3ca8c0ca7f,Cork County,MULTIPOLYGON (((-8.926795 5...,28371.98,11348791,27096.68,0.23876266,-0.9956941
4c07d11d-f564-851d-e053-ca3ca8c0ca7f,Galway County,MULTIPOLYGON (((-9.544816 5...,203424.19,81369675,43149.14,0.05302853,-1.0036785
4c07d11d-f565-851d-e053-ca3ca8c0ca7f,Galway County,MULTIPOLYGON (((-9.73048 53...,160154.42,64061769,24731.96,0.03860642,-1.0042985


# Export

In [19]:
#EXPORT
#build export path
exportPath <- paste(exportDirectory, 'imperviousSurface', sep = '/')
exportPath <- paste(exportPath, '.csv', sep = '')

smallAreasBufferExport <- as.data.frame(smallAreasSF) %>% select(1,7,8)

names(smallAreasBufferExport) <- gsub("_pct_Z","",names(smallAreasBufferExport))

write.csv(smallAreasBufferExport, exportPath, row.names = FALSE)